In [ ]:
import pandas as pd
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, RationalQuadratic, ExpSineSquared, ConstantKernel as C
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.interpolate import CubicSpline



file_path = "fm_t67_2019.fr.xlsx"
xls = pd.ExcelFile(file_path)

df = pd.read_excel(file_path, engine="openpyxl")


df.to_csv("data.csv", index=False) 
df_csv = pd.read_csv("data.csv") 


In [ ]:
def clean_mortality_table_v2(sheet_name):
    df = xls.parse(sheet_name, skiprows=4)  
    
    df.columns = df.iloc[0]  
    df = df[1:].reset_index(drop=True)  
    
    df = df.rename(columns={df.columns[0]: "Année"})
    
    return df

df_T67e = clean_mortality_table_v2("T67e")
df_T67f = clean_mortality_table_v2("T67f")
df_T67h = clean_mortality_table_v2("T67h")

df_T67e.head()


In [ ]:
df_T67f.head()

In [ ]:
df_T67h.head()

In [ ]:
columns_to_clean = [
    "moins d'un an (a)", "1 à 4 ans", "5 à 9 ans", "10 à 14 ans", "15 à 19 ans", 
    "20 à 24 ans", "25 à 29 ans", "30 à 34 ans", "35 à 39 ans", "40 à 44 ans",
    "45 à 49 ans", "50 à 54 ans", "55 à 59 ans", "60 à 64 ans", "65 à 69 ans",
    "70 à 79 ans", "80 à 89 ans", "90 à 110 ans"
]

df_T67h[columns_to_clean] = df_T67h[columns_to_clean].apply(pd.to_numeric, errors='coerce')

df_T67h["Année"] = pd.to_numeric(df_T67h["Année"], errors='coerce')

df_cleaned = df_T67h.dropna(subset=["Année"] + columns_to_clean)

df_cleaned[columns_to_clean] = df_cleaned[columns_to_clean] 

plt.figure(figsize=(12, 8))

for age_group in columns_to_clean:
    sns.lineplot(data=df_cleaned, x="Année", y=age_group, marker='o', label=age_group)

plt.title("Nombre de décés en fonction de l'année pour chaque groupe d'âge")
plt.xlabel("Année")
plt.ylabel("Taux de mortalité (%)")
plt.xticks(rotation=45)
plt.legend(title="Groupe d'âge", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

plt.show()



In [ ]:


columns_to_clean = [
    "moins d'un an (a)", "1 à 4 ans", "5 à 9 ans", "10 à 14 ans", "15 à 19 ans", 
    "20 à 24 ans", "25 à 29 ans", "30 à 34 ans", "35 à 39 ans", "40 à 44 ans",
    "45 à 49 ans", "50 à 54 ans", "55 à 59 ans", "60 à 64 ans", "65 à 69 ans",
    "70 à 79 ans", "80 à 89 ans", "90 à 110 ans"
]

df_T67h[columns_to_clean] = df_T67h[columns_to_clean].apply(pd.to_numeric, errors='coerce')

df_T67h["Année"] = pd.to_numeric(df_T67h["Année"], errors='coerce')

df_cleaned = df_T67h.dropna(subset=["Année"] + columns_to_clean)

for age_group in columns_to_clean:
    df_cleaned[f"{age_group}_survie"] = df_cleaned[age_group]  

plt.figure(figsize=(12, 8))

for age_group in columns_to_clean:
    survie_column = f"{age_group}_survie"
    sns.lineplot(data=df_cleaned, x="Année", y=survie_column, marker='o', label=f"Survie {age_group}")

plt.title("Nombre de décés en fonction de l'année pour chaque groupe d'âge")
plt.xlabel("Année")
plt.ylabel("Nombre de décés")

plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)

plt.legend(title="Groupe d'âge", bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=12)

plt.tight_layout()

plt.show()


In [ ]:
df_T67h[columns_to_clean] = df_T67h[columns_to_clean].apply(pd.to_numeric, errors='coerce')

df_T67h["Année"] = pd.to_numeric(df_T67h["Année"], errors='coerce')

df_cleaned = df_T67h.dropna(subset=["Année"] + columns_to_clean)

df_cleaned[columns_to_clean] = df_cleaned[columns_to_clean] 

plt.figure(figsize=(12, 8))

for age_group in columns_to_clean:
    sns.lineplot(data=df_cleaned, x="Année", y=age_group, marker='o', label=age_group)

plt.title("Nombre de décés en fonction de l'année pour chaque groupe d'âge")
plt.xlabel("Année")
plt.ylabel("Nombre de décés")
plt.xticks(rotation=45)
plt.legend(title="Groupe d'âge", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

plt.show()

In [ ]:

df_T67f[columns_to_clean] = df_T67f[columns_to_clean].apply(pd.to_numeric, errors='coerce')

df_T67f["Année"] = pd.to_numeric(df_T67f["Année"], errors='coerce')

df_cleaned = df_T67f.dropna(subset=["Année"] + columns_to_clean)

for age_group in columns_to_clean:
    df_cleaned[f"{age_group}_survie"] = df_cleaned[age_group]  

plt.figure(figsize=(12, 8))

for age_group in columns_to_clean:
    survie_column = f"{age_group}_survie"
    sns.lineplot(data=df_cleaned, x="Année", y=survie_column, marker='o', label=f"Survie {age_group}")

plt.title("Nombre de décés en fonction de l'année pour chaque groupe d'âge")
plt.xlabel("Année")
plt.ylabel("Nombre de décés")

plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)

plt.legend(title="Groupe d'âge", bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=12)

plt.tight_layout()

plt.show()


In [ ]:
classe_age = {
    "moins d'un an (a)": [0],
    "1 à 4 ans": [1, 2, 3, 4],
    "5 à 9 ans": [5, 6, 7, 8, 9],
    "10 à 14 ans": [10, 11, 12, 13, 14],
    "15 à 19 ans": [15, 16, 17, 18, 19],
    "20 à 24 ans": [20, 21, 22, 23, 24],
    "25 à 29 ans": [25, 26, 27, 28, 29],
    "30 à 34 ans": [30, 31, 32, 33, 34],
    "35 à 39 ans": [35, 36, 37, 38, 39],
    "40 à 44 ans": [40, 41, 42, 43, 44],
    "45 à 49 ans": [45, 46, 47, 48, 49],
    "50 à 54 ans": [50, 51, 52, 53, 54],
    "55 à 59 ans": [55, 56, 57, 58, 59],
    "60 à 64 ans": [60, 61, 62, 63, 64],
    "65 à 69 ans": [65, 66, 67, 68, 69],
    "70 à 79 ans": [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
    "80 à 89 ans": [80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
    "90 à 110 ans": [90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110]
}


In [ ]:
df_T67f[df_T67f['Année'] == 2019]



In [ ]:


def methode_naive(data, annee, classe_age):
    data_naive = data[data['Année'] == annee]
    
    ages_full = []
    survie_full = []
    
    survie_cumulative = 1
    
    for group, ages in classe_age.items():
        n = len(ages) 
        
        for i in range(n):
            taux_mortalite = data_naive[group].values[0] 
            survie_cumulative *= (1 - taux_mortalite/1000)  
            ages_full.append(ages[i]) 
            survie_full.append(survie_cumulative) 
    
    ages_full = np.array(ages_full)
    survie_full = np.array(survie_full)
    ages_fine = np.linspace(0, 110, 111)
    
    return ages_fine, survie_full

ages, survie_hommes = methode_naive(df_T67h, 1983, classe_age)
ages, survie_femmes = methode_naive(df_T67f, 1983, classe_age)

plt.figure(figsize=(12, 8))

plt.plot(ages, survie_hommes, label="Probabilité de survie pour les hommes (1983)", color='blue')

plt.plot(ages, survie_femmes, label="Probabilité de survie pour les femmes (1983)", color='red')

plt.title("Courbe de survie pour l'année 1983", fontsize=14)
plt.xlabel("Âge (en années)", fontsize=12)
plt.ylabel("Fonction de survie", fontsize=12)

plt.legend()

plt.grid(True)

plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit


def gompertz_makeham_exp(x, A, B, C):
    return A + B * np.exp(C * x)

def gompertz_makeham_puissance(x, A, B, C):
    return A + B * C ** x

def fonction_survie_exp(x, A, B, C):
    if not isinstance(x, np.ndarray):
        print("centres_age n'est pas un tableau NumPy. Conversion en tableau NumPy.")
        x = np.array(x)
    survie = np.exp(-A*x - B* np.exp(C*x)/C)  
    return np.clip(survie, 0, 1)



classes_age = [(0,0), (1, 4), (5, 9), (10, 14), (15, 19), (20, 24),
               (25, 29), (30, 34), (35, 39), (40, 44), (45, 49),
               (50, 54), (55, 59), (60, 64), (65, 69), (70, 79), (80, 89), (90, 110)]

def fonction_survie_puissance(A, B, C, x):
    """
    Calcule la fonction de survie pour le modèle de Gompertz-Makeham avec la forme de puissance.
    """
    if not isinstance(x, np.ndarray):
        print("centres_age n'est pas un tableau NumPy. Conversion en tableau NumPy.")
        x = np.array(x) 
    
    integral_mu = A * x + (B / np.log(C)) * (C**x - 1)
    
    survie = np.exp(-integral_mu)
    
    return np.clip(survie, 0, 1) 


def modele_gompertz_makeham(annee_cible, data, classe_age, type_GM): 

    donnees = data[data["Année"] == annee_cible]
    taux_mortalite = donnees.iloc[:, 2:].values.flatten()
    centres_age = [(a + b) / 2 for a, b in classes_age]
    
    if type_GM == "exp":
        p0 =  [0.60, 0.02, 0.4]   
    elif type_GM == "puissance":
        p0 = [0.01, 0.1, 1.005]  
        if type_GM == "puissance":
            bounds = ([0, 0, 1], [np.inf, np.inf, 1.5]) 
        
    ages_fit = np.linspace(0, 110, 200)

    if type_GM == "exp":
        params, covariance = curve_fit(gompertz_makeham_exp, centres_age, taux_mortalite / 1000, p0=p0, maxfev=10000)
        A, B, C = params
        mortalite_fit = gompertz_makeham_exp(ages_fit, *params)
        survie_classes = fonction_survie_exp(centres_age, A, B, C)
        survie_fit = fonction_survie_exp(ages_fit, A, B, C)
    elif type_GM == "puissance":
        params, covariance = curve_fit(gompertz_makeham_puissance, centres_age, taux_mortalite / 1000, p0=p0, bounds=bounds, maxfev=10000)
        A, B, C = params
        mortalite_fit = gompertz_makeham_puissance(ages_fit, *params)
        survie_classes = fonction_survie_puissance(A, B, C, centres_age)
        survie_fit = fonction_survie_puissance(A, B, C, ages_fit)

    return ages_fit, survie_classes, survie_fit


In [ ]:
ages,survie_classe_exp_h, survie_hommes_exp = modele_gompertz_makeham(2016, df_T67h, classe_age, "exp")
_,_, survie_femmes_exp = modele_gompertz_makeham(2016, df_T67f, classe_age, "exp")
ages,_, survie_hommes_puissance = modele_gompertz_makeham(2016, df_T67h, classe_age, "puissance")
_,_, survie_femmes_puissance = modele_gompertz_makeham(2016, df_T67f, classe_age, "puissance")

plt.figure(figsize=(12, 8))

plt.plot(ages, survie_hommes_exp, label="Hommes (modèle exponentiel)", color='blue')

plt.plot(ages, survie_femmes_exp, label="Femmes (modèle exponentiel)", color='red')

plt.plot(ages, survie_hommes_puissance, label="Hommes (modèle puissance)", color='green')

plt.plot(ages, survie_femmes_puissance, label="Femmes (modèle puissance)", color='orange')

plt.title("Courbes de survie pour l'année 2016", fontsize=14)
plt.xlabel("Âge (en années)", fontsize=12)
plt.ylabel("Fonction de survie", fontsize=12)

plt.legend()

plt.grid(True)

plt.show()

In [ ]:
def Survie_by_GP(data, classes_age, annee_cible):
    centres_age = np.array([(0.9*b + 0.1*a)  for a, b in classes_age])  
    donnees_annee = data[data["Année"] == annee_cible]
    taux_mortalite = donnees_annee.iloc[:, 2:].values.flatten()  

    taux_mortalite = pd.to_numeric(taux_mortalite, errors='coerce')  # Convertit tout en float
    taux_mortalite = np.nan_to_num(taux_mortalite, nan=0.0)  

    lambda_taux = taux_mortalite / 1000
    survie_initiale = 1.0

    survie_observee = survie_initiale * np.exp(-np.cumsum(lambda_taux * np.diff(centres_age, prepend=0)))
    X = centres_age.reshape(-1, 1)
    y = survie_observee

    kernels = [
        C(1.0, (1e-3, 1e3)) * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2)),
        C(1.0, (1e-3, 1e3)) * Matern(length_scale=1.0, length_scale_bounds=(1e-2, 1e2), nu=1.5),
        C(1.0, (1e-3, 1e3)) * RationalQuadratic(length_scale=1.0, alpha=0.1),
        C(1.0, (1e-3, 1e3)) * ExpSineSquared(length_scale=1.0, periodicity=10.0)
    ]

    best_score = -np.inf
    best_model = None
    best_kernel = None

    for kernel in kernels:
        gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)
        gp.fit(X, y)
        score = gp.log_marginal_likelihood_value_

        if score > best_score:
            best_score = score
            best_model = gp
            best_kernel = kernel

    print(f"Meilleur noyau sélectionné : {best_kernel}")
    print(f"Log-vraisemblance marginale du meilleur modèle : {best_score:.4f}")

    ages_fine = np.linspace(0, 110, 200).reshape(-1, 1)
    survie_predite, sigma = best_model.predict(ages_fine, return_std=True)

    return ages_fine, survie_predite, sigma



In [ ]:
ages_fine_femme, survie_predite_femme, sigma_femme = Survie_by_GP(df_T67f, classes_age, 1998)

ages_fine_homme, survie_predite_homme, sigma_homme = Survie_by_GP(df_T67h, classes_age, 1998)

plt.figure(figsize=(10, 6))
plt.plot(ages_fine_femme, survie_predite_femme, label="Survie interpolée (Femmes)", color="blue", linewidth=2)
plt.fill_between(
    ages_fine_femme.flatten(), 
    survie_predite_femme - 1.96 * sigma_femme, 
    survie_predite_femme + 1.96 * sigma_femme, 
    color='blue', alpha=0.2, label="Intervalle de confiance à 95% (Femmes)"
)

plt.plot(ages_fine_homme, survie_predite_homme, label="Survie interpolée (Hommes)", color="green", linewidth=2)
plt.fill_between(
    ages_fine_homme.flatten(), 
    survie_predite_homme - 1.96 * sigma_homme, 
    survie_predite_homme + 1.96 * sigma_homme, 
    color='green', alpha=0.2, label="Intervalle de confiance à 95% (Hommes)"
)

plt.xlabel("Âge")
plt.ylabel("Probabilité de survie")
plt.title("Modélisation de la fonction de survie (Femmes et Hommes, 2014)")
plt.legend()
plt.show()
